In [1]:
#Data Analyses Libraries
import pandas as pd                
import numpy as np    
from urllib.request import urlopen
import json
import glob
import os

In [2]:
#Importing Data plotting libraries
import matplotlib.pyplot as plt     
import plotly.express as px       
import plotly.offline as py       
import seaborn as sns             
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import matplotlib.ticker as ticker
import matplotlib.animation as animation

In [3]:
#Other Miscallaneous Libraries
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML
import matplotlib.colors as mc
import colorsys
from random import randint
import re

In [4]:
data = pd.read_csv(r"C:\Users\user\OneDrive\Υπολογιστής\Tests_26Apr.csv")                       
df = data.copy()

In [5]:
df.head()

,Country,Date,Tests,Positive,%,Tests /millionpeople,Positive /millionpeople,Ref.,Source_1,Source_2
0,Albania,26 Apr,7279,726,10.0,2542,254.0,cite_note-127,https://new.shendetesia.gov.al/covid-19-minist...,
1,Argentina,20 Apr,36611,3031,8.3,812,67.0,cite_note-128,https://www.argentina.gob.ar/coronavirus/infor...,
2,Armenia,25 Apr,17342,1677,9.7,5875,568.0,cite_note-129,https://ncdc.am/coronavirus/confirmed-cases-by...,
3,Australia,24 Apr,482420,6676,1.4,18993,263.0,"cite_note-Australian_Government,_DOH-130",https://twitter.com/Gajafar/status/12536561297...,
4,Austria,26 Apr,227631,15140,6.7,25569,1701.0,cite_note-131,https://www.sozialministerium.at/Informationen...,


In [6]:


country_list = data['Country'].unique()

country_grouped_covid = data[0:1]

for country in country_list:
    test_data = data['Country'] == country   
    test_data = data[test_data]
    country_grouped_covid = pd.concat([country_grouped_covid, test_data], axis=0)
    
country_grouped_covid.reset_index(drop=True)
country_grouped_covid.head()

#Dropping of the column Last Update
country_grouped_covid.drop('Ref.', axis=1, inplace=True)
country_grouped_covid.drop('Source_1', axis=1, inplace=True)
country_grouped_covid.drop('Source_2', axis=1, inplace=True)

#Printing the dataset
country_grouped_covid.head()

#country_grouped_covid holds the dataset for the country

,Country,Date,Tests,Positive,%,Tests /millionpeople,Positive /millionpeople
0,Albania,26 Apr,7279,726,10.0,2542,254.0
0,Albania,26 Apr,7279,726,10.0,2542,254.0
1,Argentina,20 Apr,36611,3031,8.3,812,67.0
2,Armenia,25 Apr,17342,1677,9.7,5875,568.0
3,Australia,24 Apr,482420,6676,1.4,18993,263.0


In [7]:
#Creating a dataset to analyze the cases country wise - As of 04/26/2020

latest_data = country_grouped_covid['Date'] == '04/26/2020'
country_data = country_grouped_covid[latest_data]

#The total number of reported Countries
country_list = country_data['Country'].unique()
print("The total number of countries with COVID-19 Confirmed cases = {}".format(country_list.size))

The total number of countries with COVID-19 Confirmed cases = 0


In [8]:
#I have a problem with the form of the date

In [9]:
#Creating the interactive map
py.init_notebook_mode(connected=True)

#GroupingBy the dataset for the map
formated_gdf = data.groupby(['Date', 'Country'])['Tests', 'Positive'].max()
formated_gdf = formated_gdf.reset_index()
formated_gdf['Date'] = pd.to_datetime(formated_gdf['Date'])
formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d')

formated_gdf['log_ConfirmedCases'] = np.log(formated_gdf.Confirmed + 1)
formated_gdf['log_Fatalities'] = np.log(formated_gdf.Deaths + 1)

#Plotting the figure


fig = px.choropleth(formated_gdf, locations="Country", locationmode='country names', 
                     color="log_ConfirmedCases", hover_name="Country",projection="mercator",
                     animation_frame="Date",width=1000, height=800,
                     color_continuous_scale=px.colors.sequential.Viridis,
                     title='The Spread of COVID-19 Cases Across World')

#Showing the figure
fig.update(layout_coloraxis_showscale=True)
py.offline.iplot(fig)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-04-13 00:00:00

In [10]:
#Plotting a bar graph for positive cases vs tests due to COVID-19 in World.

unique_dates = country_grouped_covid['Date'].unique()
Tests = []
Positive = []


for date in unique_dates:
    date_wise = country_grouped_covid['Date'] == date  
    test_data = country_grouped_covid[date_wise]
    
    Tests.append(test_data['Tests'].sum())
    Positive.append(test_data['Positive'].sum())
    
    #Converting the lists to a pandas dataframe.

country_dataset = {'Date' : unique_dates, 'Tests' : Tests, 'Positive' : Positive}
country_dataset = pd.DataFrame(country_dataset)

#Plotting the Graph of Cases vs tests Globally.

fig = go.Figure()
fig.add_trace(go.Bar(x=country_dataset['Date'], y=country_dataset['Positive'], name='Positive Cases of COVID-19', marker_color='rgb(55, 83, 109)'))
fig.add_trace(go.Bar(x=country_dataset['Date'],y=country_dataset['Tests'],name='Total Tests conducted for COVID-19',marker_color='rgb(26, 118, 255)'))

fig.update_layout(title='Confirmed Cases and Deaths from COVID-19',xaxis_tickfont_size=14,
                  yaxis=dict(title='Reported Numbers',titlefont_size=16,tickfont_size=14,),
    legend=dict(x=0,y=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'),barmode='group',bargap=0.15, bargroupgap=0.1)
fig.show()



In [22]:
#scatter plot
#data=data.groupby(['Country', 'Date', 'Tests', 'Positive', '%'],
             #as_index=False)

temp = data[Positive!=0]    
temp = data[Tests!=0]

pop_density = covid_indicators[["population_density", "country"]]
pop_density = pop_density.replace("Korea Republic of", "South Korea")
temp = temp.merge(pop_density, left_on=["entity"], right_on=["country"])
temp = temp.drop(["country"],axis=1)

temp["tests_by_cases"] = (temp["Tests"] / temp["Positive"])


def draw_scatter(temp, text):
    # visualization
    chart = alt.Chart(temp).mark_circle().encode(
        alt.X('Tests', scale=alt.Scale(zero=False)),
        alt.Y('Positive', scale=alt.Scale(zero=False, padding=1)),
        color=alt.Color('population_density:Q', scale=alt.Scale(scheme="viridis")),
        size=alt.Size('tests_by_cases', scale=alt.Scale(domain=[-1, 100])),
        #size='tests_by_cases',
        tooltip=['tests_by_cases:Q',
             'Country:N',
             'Tests:Q',
             'Positive:Q',
             'Date:T']
    ).configure_mark(
        size=50
    ).properties(
        title="Tests Conducted vs Positive Cases - "+text,
        width=600,
        height=300
    ).configure_axis(
        grid=False
    ).configure_view(
        strokeWidth=0
    ).configure_title(
        fontSize=30,
        font='Agency FB',
        anchor='start',
        color='black'
    )
    
    return chart

KeyError: 'Column not found: True'

In [24]:
def tests_and_cases(Country):
    """
    country : Choose the country
    """
    alt.data_transformers.enable('default', max_rows=None)

    """
    input_dropdown = alt.binding_select(options=['Italy', 'China'])
    selection = alt.selection_single(fields=['entity'], bind=input_dropdown, name='Country of ')
    color = alt.condition(selection,
                    alt.Color('entity:N', legend=None),
                    alt.value('lightgray'))
    """

  

 # Tests Performed
    tests = list(data[data.Tests!=0].Tests.sort_values(ascending=False))[0]
    stamp = list(data[data.Tests!=0].Date.sort_values(ascending=False))[0]
    month = (stamp.strftime("%B")+" "+stamp.strftime("%d"))

    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')

    graph = alt.Chart().mark_line().encode(
        x='Date:T',
        y='Positive:Q',
        #color=color,
        tooltip=['Date:T', 'Positive:Q']
    ).transform_timeunit(
        month='monthdate(Date)'
    ).properties(
        title=("Confirmed Cases in "+Country+" (Number of Tests: "+str(int(Tests))+")"+" (Recorded: "+month+")"),
        width=600,
        height=300
    )

    selectors = alt.Chart().mark_point().encode(
        x='Date:T',
        opacity=alt.value(0),
    ).add_selection(
        nearest
    )

    points = graph.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    text = graph.mark_text(align='left', dx=-15, dy=-10).encode(
        text=alt.condition(nearest, 'Positive:Q', alt.value(' '))
    )

    rules = alt.Chart().mark_rule(color='gray').encode(
        x='Date:T',
    ).transform_filter(
        nearest
    )

    # Combine all layers
    chart = alt.layer(graph, selectors, points, rules, text,
                       data=dc)

    chart.configure_title(
        fontSize=30,
        font='Agency FB',
        anchor='start',
        color='gray'
    )
    
    return chart

In [25]:
tests_and_cases("China")

NameError: name 'alt' is not defined